In [ ]:
import pandas as pd
from gensim.models import word2vec, FastText
import pandas as pd
import re
from numpy import dot
from numpy.linalg import norm
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import spacy_sentence_bert
import spacy_transformers

In [ ]:
nlp = spacy_sentence_bert.load_model('en_stsb_bert_large')
df3 = pd.read_csv("llama_bertopic_region.csv")

k = []
for i in df3.Representation.values:
    p = i[2:-2]
    k.append(p.split(','))
    
    
c = []

for j in range(len(k)):
    clean = []

    for i in range(len(k[j])):

        if i == 0:
            clean.append(k[j][i][0:-1])
        else:
            clean.append(k[j][i][2:-1])
    c.append(clean)

In [ ]:
dff1 =  pd.read_csv("Female_aligned_data.csv")
dff2 =  pd.read_csv("Male_aligned_data.csv")

df = pd.concat([dff1, dff2], ignore_index=True)


import re 
clean_txt = []
for w in range(len(df['0'])):
    desc = df['0'][w].lower()
    desc = re.sub('[^a-zA-Z]', ' ', desc) # punc
    desc=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",desc) #tags
    desc=re.sub("(\\d|\\W)+"," ",desc)
    clean_txt.append(desc)
    
c_sum = sum(c, [])

In [ ]:
vec = []
tex = []
cnt = 0
for i in clean_txt:
    if any(ext in str(i) for ext in c_sum):
#         print(i)
        print(cnt)
        tokens = nlp(str(i))

        for token in tokens:
            vec.append(token.vector)
            tex.append(token.text)
        cnt = cnt + 1
        
        
dfff = pd.DataFrame(
    {'Text': tex,
     'Vector': vec
    })


vec_mean = []
token = []
for j in c_sum:
    vec_mean.append(dfff[dfff.Text == j].Vector.mean())
    token.append(j)   
    
    
dfff_mean = pd.DataFrame(
    {'Word': token,
     'Vector': vec_mean
    })

# dfff_mean.to_csv("vectors_regiona_stsbbertlarge_updated.csv")

In [ ]:
she = []
he = []
cnt = 0
for i in clean_txt:
    if 'she' in i:
#     print(i)
        print(cnt)
        tokens = nlp(str(i))

        for token in tokens:
            if token.text == 'she':
                she.append(token.vector)
    if 'he' in i:

        tokens = nlp(str(i))

        for token in tokens:
            if token.text == 'he':
                he.append(token.vector)
                
        cnt = cnt + 1
        
        
meanhe = sum(he) / len(he)
meanshe = sum(she) / len(she)

t = ['he', 'she']
means = [meanhe, meanshe]

dfff_heshe = pd.DataFrame(
    {'Word': t,
     'Vector': means
    })


In [ ]:
dfff_mean = dfff_mean[dfff_mean['Vector'].notna()]


avg = []
for i in range(len(c)):
    ind = []
#     print(c[i])
    for k in c[i]:
#         print(k)
        if k in dfff_mean.Word.unique():
            ind.append(dfff_mean[dfff_mean.Word == k].Vector.tolist()[0])
    avg.append(sum(ind)/len(ind))

In [ ]:
female = []
male = []

for k in range(len(c)):
    m1 = avg[k]
    type(m1)
    q = dfff_heshe[dfff_heshe.Word == 'she'].Vector.tolist()[0]
    cos_sim = dot(q, m1)/(norm(q)*norm(m1))
    female.append(cos_sim)

for k in range(len(c)):
    m1 = avg[k]
    type(m1)
    q = dfff_heshe[dfff_heshe.Word == 'he'].Vector.tolist()[0]
    cos_sim = dot(q, m1)/(norm(q)*norm(m1))
    male.append(cos_sim)


more = []
# female_more = []
for i,j in zip(male, female):
    if i < j:
        more.append('female')
    else:
        more.append('male')
        
        
 # [abs(a_i - b_i) for a_i, b_i in zip(female, male)]

for i in male:
    for j in female:
        if abs(i - j) < 0.01 and male.index(i) != female.index(j) and ((more[male.index(i)] =='male' and more[female.index(j)] == 'female') or (more[male.index(i)] =='female' and more[female.index(j)] == 'male')) :
      # print(abs(i - j))
      # print(i, j)
            print(male.index(i), female.index(j))

In [ ]:
fin = pd.DataFrame(list(zip(male, female, more, df3.Representation.values, df3.Llama2.values)),
               columns =['Similarity to he', 'Similarity to she', 'Closer', 'Topic Words', 'Llama2'])

fin.to_csv("Regiona_stsb_BERTembeddings_distance.csv")